In [1]:
import pandas as pd

In [2]:
df_original_patient_info = pd.read_csv('Patient_info_original.csv') # Original patient info file

In [3]:
df_glucose_measurements = pd.read_csv('../T1DiabetesGranada/Glucose_measurements.csv') # Final Glucose_measurements.csv file
df_biochemical_parameters = pd.read_csv('../T1DiabetesGranada/Biochemical_parameters.csv') # Final Biochemical_parameters.csv file
df_diagnostics = pd.read_csv('../T1DiabetesGranada/Diagnostics.csv') # Final Diagnostics.csv file

# Get variables from ***Glucose_measurements.csv***

In [ ]:
df_custom_info_glucose = pd.DataFrame(columns=['Patient_ID', 'Initial_measurement_date', 'Final_measurement_date', 'Number_of_days_with_measures', 'Number_of_measurements']) # Dataframe to storage the extracted variables

number_of_days = 0
number_of_measurements = 0
current_ID = df_glucose_measurements.iloc[0]['Patient_ID'] # First patient ID
current_date = df_glucose_measurements.iloc[0]['Measurement_date'] # First date
current_row = df_glucose_measurements.iloc[0] # First row

series_current_patient = pd.Series(index=df_custom_info_glucose.columns, dtype=object) # Serie that will storage the info of the current patient

index = 0
size_df = df_glucose_measurements.shape[0]
patient_number = 0
while index < size_df - 1:

    series_current_patient['Patient_ID'] = current_row['Patient_ID']
    series_current_patient['Initial_date'] = current_row['Measurement_date']
    start_index = index
    patient_number += 1
    print('New Patient_ID: ', patient_number, ' -> ',  current_row['Patient_ID'], 'At index: ', index) # Output to get process update

    while current_ID == current_row['Patient_ID']: # If it is the same patient

        if current_date != current_row['Measurement_date']: # Is a new day compare with the date of the previous row
            number_of_days += 1

        current_date = current_row['Measurement_date'] # Update current date with the current value

        # Update values
        if index + 1 < df_glucose_measurements.shape[0]: # if there is not in the end of the data
            index += 1
        else:
            break

        current_row = df_glucose_measurements.iloc[index] # Update the row

    # New patient (new Patient_ID)
    # Update the Series from the past patient
    current_ID = df_glucose_measurements.iloc[index]['Patient_ID'] # New Patient_ID

    current_number_of_measures = (index - 1) - start_index + 1 # (index - 1) is the last index of the previous patient. 1 is added to be accurate (if start index is 2000 and end index is 2005, end_index - start_index [2005 - 2000] = 5, but there is 6 elements [2000, 2001, 2002, 2003, 2004, 2005])
    custom_end_date = df_glucose_measurements.iloc[index - 1]['Measurement_date'] # The date of the las reading of the patient

    series_current_patient['End_date'] = custom_end_date
    series_current_patient['Number_of_days_with_measures'] = number_of_days
    number_of_days = 0
    series_current_patient['Number_of_measurements'] = current_number_of_measures

    df_custom_info_glucose = pd.concat([pd.DataFrame([series_current_patient]), df_custom_info_glucose]) # Add to the dataframe the new info of the current patient

df_custom_info_glucose.reset_index(inplace=True, drop=True) # To set proper index to the dataframe

print(df_custom_info_glucose.info())

# Get variables from ***Biochemical_parameters.csv***

In [ ]:
df_custom_info_bioch_params = pd.DataFrame(columns=['Patient_ID', 'Initial_biochemical_parameters_date', 'Final_biochemical_parameters_date', 'Number_of_biochemical_parameters']) # Dataframe to storage the extracted variables

number_of_measurements = 0
current_ID = df_biochemical_parameters.iloc[0]['Patient_ID'] # First patient ID
current_date = df_biochemical_parameters.iloc[0]['Reception_date'] # First date
current_row = df_biochemical_parameters.iloc[0] # First row

series_custom_demographics_bioch_params = pd.Series(index=df_custom_info_bioch_params.columns, dtype=object) # Serie that will storage the info of the current patient

index = 0
size_df = df_biochemical_parameters.shape[0]
patient_number = 0
while index < size_df - 1:

    series_custom_demographics_bioch_params['Patient_ID'] = current_row['Patient_ID']
    series_custom_demographics_bioch_params['Initial_biochemical_parameters_date'] = current_row['Reception_date']
    start_index = index
    patient_number += 1
    print('New Patient_ID: ', patient_number, ' -> ',  current_row['Patient_ID'], 'At index: ', index) # Output to get process update

    while current_ID == current_row['Patient_ID']: # If it is the same patient

        current_date = current_row['Reception_date'] # Update current date with the current value

        # Update values
        if index + 1 < df_biochemical_parameters.shape[0]: # if there is not in the end of the data
            index += 1
        else:
            break

        current_row = df_biochemical_parameters.iloc[index]

    # New patient Patient_ID
    # Update the Series from the past patient
    current_ID = df_biochemical_parameters.iloc[index]['Patient_ID'] # New Patient_ID

    current_number_of_bioch_params = (index - 1) - start_index + 1 # (index - 1) is the last index of the previous patient. 1 is added to be accurate (if start index is 2000 and end index is 2005, end_index - start_index [2005 - 2000] = 5, but there is 6 elements [2000, 2001, 2002, 2003, 2004, 2005])
    custom_end_date = df_biochemical_parameters.iloc[index - 1]['Reception_date'] # The date of the las reading of the patient

    series_custom_demographics_bioch_params['Final_biochemical_parameters_date'] = custom_end_date
    series_custom_demographics_bioch_params['Number_of_biochemical_parameters'] = current_number_of_bioch_params
    df_temp_custom_demographics_bioch_params = pd.DataFrame([series_custom_demographics_bioch_params])
    df_custom_info_bioch_params = pd.concat([df_custom_info_bioch_params, df_temp_custom_demographics_bioch_params])

df_custom_info_bioch_params.reset_index(inplace=True, drop=True)

print(df_custom_info_bioch_params.info())


# Get variable from ***Diagnostic.csv***

In [ ]:
series_number_of_diagnostics_by_patient = df_diagnostics['Patient_ID'].value_counts() # The number of times that a Patient_ID appear on the file represent the number of diagnostics made to that patient.
df_custom_info_diagnostics = pd.DataFrame(data=series_number_of_diagnostics_by_patient.values, index=series_number_of_diagnostics_by_patient.index)
df_custom_info_diagnostics.reset_index(inplace=True)
df_custom_info_diagnostics.columns = ['Patient_ID', 'Number_of_diagnostics']
df_custom_info_diagnostics.sort_values(by=['Patient_ID'], inplace=True)
df_custom_info_diagnostics.reset_index(inplace=True, drop=True)
print(df_custom_info_diagnostics.info())
df_custom_info_diagnostics.head()

# Add new variables from ***Glucose_measurements.csv*** to the original Patient_info file

In [7]:
df_new_patient_info = df_original_patient_info.copy() # A copy of the original file

In [ ]:
# IDs from original Patient_info file
ID_original_info = df_new_patient_info['Patient_ID'].unique() # Patient_ID from the original info file
ID_original_info = pd.Series(data=ID_original_info) # This operation it is necessary to get a Series instead of a numpy array

# IDs from glucose info dataframe
ID_custom_from_glucose = df_custom_info_glucose['Patient_ID'].unique() # Patient_ID from glucose variables dataframe
ID_custom_from_glucose = pd.Series(data=ID_custom_from_glucose) # This operation it is necessary to get a Series instead of a numpy array

# The index of the elements of the intersection are could be not the same on each file, for that to access to each file is needed to find both intersections separately.
ID_custom_from_glucose_INTERSECTION_original= ID_custom_from_glucose[ID_custom_from_glucose.isin(ID_original_info)]
ID_original_INTERSECTION_custom_from_glucose = ID_original_info[ID_original_info.isin(ID_custom_from_glucose)]

# Index
index_custom_from_glucose = ID_custom_from_glucose_INTERSECTION_original.index
index_original_info = ID_original_INTERSECTION_custom_from_glucose.index

# Assigning values
df_new_patient_info.loc[index_original_info, 'Initial_measurement_date'] = df_custom_info_glucose.loc[index_custom_from_glucose, 'Initial_date'].values
df_new_patient_info.loc[index_original_info, 'Final_measurement_date'] = df_custom_info_glucose.loc[index_custom_from_glucose, 'End_date'].values
df_new_patient_info.loc[index_original_info, 'Number_of_days_with_measures'] = df_custom_info_glucose.loc[index_custom_from_glucose, 'Number_of_days_with_measures'].values
df_new_patient_info.loc[index_original_info, 'Number_of_measurements'] = df_custom_info_glucose.loc[index_custom_from_glucose, 'Number_of_measurements'].values

print(df_new_patient_info.info())
df_new_patient_info.head()

# Add new variables from ***Biochemical_parameters.csv*** to the original Patient_info file

In [ ]:
# IDs from biochemical parameters info dataframe
ID_custom_from_bioch_params = df_custom_info_bioch_params['Patient_ID'].unique() # Patient_ID from glucose variables dataframe
ID_custom_from_bioch_params = pd.Series(data=ID_custom_from_bioch_params) # This operation it is necessary to get a Series instead of a numpy array

# The index of the elements of the intersection are could be not the same on each file, for that to access to each file is needed to find both intersections separately.
ID_custom_from_bioch_params_INTERSECTION_original= ID_custom_from_bioch_params[ID_custom_from_bioch_params.isin(ID_original_info)]
ID_original_INTERSECTION_custom_from_bioch_params = ID_original_info[ID_original_info.isin(ID_custom_from_bioch_params)]

# Index
index_custom_from_bioch_params = ID_custom_from_bioch_params_INTERSECTION_original.index
index_original_with_bioch_params = ID_original_INTERSECTION_custom_from_bioch_params.index

# Assigning values
df_new_patient_info.loc[index_original_with_bioch_params, 'Initial_biochemical_parameters_date'] = df_custom_info_bioch_params.loc[index_custom_from_bioch_params, 'Initial_biochemical_parameters_date'].values
df_new_patient_info.loc[index_original_with_bioch_params, 'Final_biochemical_parameters_date'] = df_custom_info_bioch_params.loc[index_custom_from_bioch_params, 'Final_biochemical_parameters_date'].values
df_new_patient_info.loc[index_original_with_bioch_params, 'Number_of_biochemical_parameters'] = df_custom_info_bioch_params.loc[index_custom_from_bioch_params, 'Number_of_biochemical_parameters'].values

print(df_new_patient_info.info())
df_new_patient_info.head()

# Add new variables from ***Diagnostic.csv*** to the original Patient_info file

In [ ]:
# IDs from diagnostic info dataframe
ID_custom_from_diagnostics = df_custom_info_diagnostics['Patient_ID'].unique()
ID_custom_from_diagnostics = pd.Series(data=ID_custom_from_diagnostics) # This operation it is necessary to get a Series instead of a numpy array

# The index of the elements of the intersection are could be not the same on each file, for that to access to each file is needed to find both intersections separately.
ID_custom_from_diagnostics_INTERSECTION_original = ID_custom_from_diagnostics[ID_custom_from_diagnostics.isin(ID_original_info)]
ID_original_INTERSECTION_diagnostics = ID_original_info[ID_original_info.isin(ID_custom_from_diagnostics)]

# Index
index_custom_from_diagnostics_INTERSECTION_original = ID_custom_from_diagnostics_INTERSECTION_original.index
index_original_INTERSECTION_diagnostics = ID_original_INTERSECTION_diagnostics.index

# Assigning value
df_new_patient_info.loc[index_original_INTERSECTION_diagnostics, 'Number_of_diagnostics'] = df_custom_info_diagnostics.loc[index_custom_from_diagnostics_INTERSECTION_original, 'Number_of_diagnostics'].values

print(df_new_patient_info.info())
df_new_patient_info.head()